# Importing data + Setup

In [ ]:
import os
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', None)          # stops pandas from truncating columns

In [ ]:
print(os.listdir("datasets/"))  
openstackDataset = pd.read_csv("datasets/openstack_metrics.csv")
qtDataset = pd.read_csv("datasets/qt_metrics.csv")

In [ ]:
# pick datasets for training and inference testing, this allows for easy switching between them
trainingDataset = openstackDataset
inferenceDataset = qtDataset

In [ ]:
# removing unnecessary attributes early since they hinder analysis-functions like correlation matrices
trainingDataset = trainingDataset.drop(['commit_id', 'author_date'], axis=1)   
inferenceDataset = inferenceDataset.drop(['commit_id', 'author_date'], axis=1) 

# Understanding the data

### Understanding each feature of the datasets

In [ ]:
#0 commit_id :      unique id for the commited code                                #16 ndev :   N of devs that have modied the file in the past
#1 author_date :    id for the                                                     #17 age :    time from previous update to the file
#2 bugcount :       amount of bugs detected (previous bugs? -> rereviews)          #18 nuc :    N of prior(unique) changes to the file
#3 fixcount :       amount of fixes                                                #19 app :    N of reviewers who voted on integration
#4 la :             lines added                                                    #20 aexp :   N of changes author has participated in
#5 ld :             lines deleted                                                  #21 rexp :   aexp but reviewer instead of author
#6 nf :             modified files                                                 #22 oexp :   o?
#7 nd :             modified directories                                           #23 arexp :  aexp but weighted by recency of changes
#8 ns :             modified subsystems                                            #24 rrexp :  arexp but reviewer instead of author
#9 ent :            entropy, spread of modified lines across files                 #25 orexp :  o?
#10 revd :          reviewed/revised? revd=False -> rxxx=NaN                       #26 asexp :  N of changes within the subsystem from the author
#11 nrev :          N of revisions to the commit                                   #27 rsexp :  asexp but reviewer instead of author
#12 rtime :         time between commit and approval                               #28 osexp :  o?
#13 tcmt :          total comments on the commit?                                  #29 asawr :  Proportion of previous changes to the subsystem from the author
#14 hcmt :          N of non-automated comments during review                      #30 rsawr :  asawr but reviewer instead of author
#15 self :          only self-checked? inverse of revd                             #31 osawr :  o?

In [ ]:
trainingDataset.sample(3)

In [ ]:
inferenceDataset.sample(3)

### Visualising and analysing the datasets

In [ ]:
print("Shape of training dataset:", trainingDataset.shape)
print("Shape of training dataset:", inferenceDataset.shape)

In [ ]:
#data.isna().sum()           # sums the amount of NaN values for each attribute
#data.describe()             # basic statistics for each attribute
#data.nunique()              # amount of unique values for each attribute
trainingDataset.info()

In [ ]:
inferenceDataset.info()

In [ ]:
print("Amount of instances where the commit was reviewed/revised in training dataset:", trainingDataset.revd.value_counts())
print("Amount of instances where the commit was reviewed/revised in inference dataset:", inferenceDataset.revd.value_counts())

In [ ]:
print("Spread of bugs found (does not include instances of NaN)")
sns.countplot(x='bugcount', data=trainingDataset)             

In [ ]:
print("Amount of instances of bugcount containing NaN:", trainingDataset.bugcount.isna().sum())
print("Amount of times bugs were detected:", trainingDataset.bugcount.value_counts().sum() - trainingDataset.bugcount.value_counts().get(0.0),
                    "out of", trainingDataset.bugcount.count() + trainingDataset.bugcount.isna().sum())

### Analysing correlation between features

In [ ]:
correlation_matrix = trainingDataset.corr()
high_correlations = correlation_matrix[(correlation_matrix > 0.9) | (correlation_matrix < -0.9)]
# Replace NaN values with an empty string
high_correlations.fillna('', inplace=True)

# Drop correlation measures for columns with themselves
for col in high_correlations.columns:
    high_correlations.loc[col, col] = ''
# Print correlation for each combination of features
for col in high_correlations.columns:
    correlations = high_correlations[col][high_correlations[col] != '']
    if not correlations.empty:
        print(f"Correlations with {col}:")
        print(correlations)
        print()

In [ ]:
trainingDataset.corr()['revd']

# Cleaning the data

### Notes on replacing NaN values

Instances of NaN in bugcount/fixcount are assumed to imply that 0 bugs/fixes were recorded.

Based on correlation analysis of ['revd'], the remaining instances of NaN are all from features related to reviews/revisions of the commit
 
These instances are NaN iff ['revd'] = False, implying no review was made to the commit -> all review related features are therefore NaN

Replacing NaN with 0 is not equally suitable for all of these features, but is done for simplicity, example:

* ['app'] = 0, implies that no reviewers voted on integration when there was no review to vote on, this seems suitable

* ['aexp'] = 0, implies the author has participated in 0 previous changes, which is not necessarily true, not so suitable

Given more time, this could be better adapted and customized to the dataset

### Cleaning of training dataset

In [ ]:
# Removing all "r" and "o" feature-types
# they contain highly correlated data, increases dimensional complexity, and rank very low on feature importance when training
unnecessaryColumns = ['arexp', 'self', 'revd', 'rexp', 'rrexp', 'rsexp', 'rsawr', 'oexp', 'orexp', 'osexp', 'osawr']
trainingDataset = trainingDataset.drop(unnecessaryColumns, axis=1)

# For the remaining features containing NaN, NaN are assumed to imply 0, see argument above
trainingDataset = trainingDataset.fillna(0.0)              

# turn bugcount into binary feature instead of multi-class 
trainingDataset['bugcount'] = trainingDataset['bugcount'].apply(lambda x: 1 if x != 0 else 0)

### Repeated cleaning for inference dataset

In [ ]:
inferenceDataset = inferenceDataset.drop(unnecessaryColumns, axis=1)
inferenceDataset = inferenceDataset.fillna(0.0) 

inferenceDataset['bugcount'] = inferenceDataset['bugcount'].apply(lambda x: 1 if x != 0 else 0)

# extract target feature ['bugcount'] for inference testing
inference_bugcount = inferenceDataset.bugcount     
inference_set = inferenceDataset.drop(['bugcount'], axis=1) 

# Modelling

In [31]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

#### Synthetic Minority Oversampling Technique (SMOTE)

Bugcount is vastly imbalanced, this is solved by increasing the ratio of instances with found bugs. 

* Oversampling: SMOTE chooses random samples of minority class, finds similar instances of data using k-nearest neighbor and generates a new instance between the original sample and one random neighbor.

* Undersampling: Randomly undersamples data with majority class, in this case: bugcount = 0

In [32]:
# import data and extract target feature before balancing 
training_data = trainingDataset
target_feature = training_data.bugcount
training_data = training_data.drop(['bugcount'], axis=1)

# show class ratio
print(target_feature.value_counts())

# prepare pipeline for balancing
oversample = SMOTE(sampling_strategy=0.3)                   
undersample = RandomUnderSampler(sampling_strategy=0.5)
steps = [('over', oversample), ('under', undersample)]
pipeline = Pipeline(steps=steps)

# balance bugcount class
training_data, target_feature = pipeline.fit_resample(training_data, target_feature)
# show new balanced class ratio
print(target_feature.value_counts())

bugcount
0    24887
1     1968
Name: count, dtype: int64
bugcount
0    14932
1     7466
Name: count, dtype: int64


## Training and validation

#### Standardizing features and splitting datasets

In [33]:
# splitting dataset into test and train sets
x_train, x_test, y_train, y_test = train_test_split(training_data, target_feature, test_size=0.1, random_state=100)

In [34]:
# Standardizing features to values [-1, 1], can help some models accurately depict features during training
# Example: SVM models are sensitive to the scale of features, especially SVMs with linear kernels. 
# This also greatly reduces training times, especially for SVM models
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

x_train_standardized = sc.fit_transform(x_train)
x_test_standardized = sc.transform(x_test)

#### RandomForestClassifier model, validated using k-fold Cross Validation

In [35]:
# specify classifier model
rfc_model = RandomForestClassifier(n_estimators=25)     #25->100, marginally better but large impact on compute

# define k-fold strategy and perform crossvalidation
crossValidation = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)     # 10 splits is a good bias-variance balance
scores = cross_val_score(rfc_model, training_data, target_feature, scoring='f1_macro', cv=crossValidation, n_jobs=-1)   
print('CrossVal mean f1_macro: %.3f' % (np.mean(scores)))         # f1_macro scores adds weights to results from each class

# train RandomForestClassifier model
rfc_model.fit(x_train, y_train)

# inference on test set
rfc_predictions = rfc_model.predict(x_test)
print("Trained model f1-score: %.3f" % (f1_score(y_test, rfc_predictions)))

CrossVal mean f1_macro: 0.885
Trained model f1-score: 0.856


#### Support Vector Classifier model

In [36]:
svm_model = SVC(kernel='linear')        # kernel alternatives: 'rbf', 'poly' and 'sigmoid'
svm_model.fit(x_train_standardized, y_train)
svm_predictions = svm_model.predict(x_test_standardized)
svm_f1 = f1_score(y_test, svm_predictions)
print("SVM f1 score:", svm_f1)

SVM f1 score: 0.5499600319744204


#### Naive Bayes model, GaussianNB

In [37]:
# This type of model also performs much better using standardized features
nb_model = GaussianNB()
nb_model.fit(x_train_standardized, y_train)
nb_predictions = nb_model.predict(x_test_standardized)
nb_f1 = f1_score(y_test, nb_predictions)
print("Naive Bayes f1 score:", nb_f1)

Naive Bayes f1 score: 0.4939467312348668


# Results

#### Model performance from training

In [ ]:
# y_pred for each model: rfc_predictions, svm_predictions or nb_predictions
print(classification_report(y_test, rfc_predictions, target_names=['no bugs', 'contains bugs']))

In [ ]:
ConfusionMatrixDisplay.from_estimator(rfc_model, x_test, y_test) 

#### Model performance on inference dataset

In [ ]:
# Model alternatives: rfc_model, svm_model or nb_model
# Note: Predictions by svm_model and nb_model might give warnings since inference_set is not standardized. 
inference_results = rfc_model.predict(inference_set)
print(classification_report(inference_bugcount, inference_results, target_names=['no bugs', 'contains bugs'])) 

# Analysis

#### Calculate baseline: what are the odds of finding a bug in general?

In [ ]:
# Baseline prediction for different datasets: training_data -> target_feature | inference_data -> inference_bugcount 
count_zero = inference_bugcount.value_counts().get(0)
count_one = inference_bugcount.value_counts().get(1)
totalCount = count_one + count_zero
chance_zero = count_zero / totalCount
chance_one = count_one / totalCount
randomGuessAccuracy = (chance_zero + chance_one) / 2
weightedGuessAccuracy = (chance_one * chance_one) + (chance_zero * chance_zero)
print("Bugcount values, 0: %d, 1: %d" %(count_zero, count_one))
print("Probability of majority class, p(0): %.3f" %(chance_zero))
print("Propability of minority class, p(1): %.3f" %(chance_one))
print("Accuracy of a random guess: %.3f" % randomGuessAccuracy)
print("Accuracy of a weighted guess: %.3f" % weightedGuessAccuracy)

#### Feature importance from training

In [ ]:
feature_importance = pd.Series(rfc_model.feature_importances_, index=x_train.columns).sort_values(axis = 0, ascending = False)
feature_importance